In [0]:
# Code Author:-
# Name: Shivam Gupta
# Net ID: SXG190040
# CS 6350.001 - Big Data Management and Analytics - F20 Assignment 3

In [0]:
from math import sqrt
from numpy import array
import pandas as pd
from pyspark.sql.window import Window
from pyspark.sql.functions import col, rank
from pyspark.mllib.clustering import KMeans


Item_user_Data = sc.textFile("/FileStore/tables/itemusermat")
Movies_Data_DF = spark.read \
    .option("sep", "::") \
    .csv("/FileStore/tables/movies.dat") \
    .withColumnRenamed("_c0", "Movie_ID") \
    .withColumnRenamed("_c1", "Movie_Title") \
    .withColumnRenamed("_c2", "Movie_Genre") \
    .dropDuplicates()
Ratings_Data = Item_user_Data.map(lambda Line_Data: array([int(item) for item in Line_Data.split(' ')[1:]]))
Clusters_Num = 10
Kmeans_Model = KMeans.train(Ratings_Data, Clusters_Num, maxIterations=10, initializationMode="random")
 
def Kmeans_Cluster_Predict(Data_Points):
    Centroid = Kmeans_Model.predict(Data_Points[1:])
    return (Data_Points[0], Centroid)
 
Cluster_INFO = Item_user_Data.map(lambda Line_Data: array([int(item) for item in Line_Data.split(' ')])).map(lambda Data_Points: Kmeans_Cluster_Predict(Data_Points)).collect()
output_Columns = ["MovieID","Cluster_Num"]
output_df = pd.DataFrame(list(Cluster_INFO), columns = output_Columns)
Res_DF = spark.createDataFrame(output_df)
 
Results_DF = Res_DF.join(Movies_Data_DF, Movies_Data_DF.Movie_ID == Res_DF.MovieID, how="inner")
 
Partition_Window = Window.partitionBy(Results_DF['Cluster_Num']).orderBy(Results_DF['MovieID'].desc())
  
Top5_Results_DF = Results_DF.select('*', rank().over(Partition_Window).alias('rank')).filter(col('rank') <= 5)
Top5_Results_DF.show()

+-------+-----------+--------+--------------------+--------------------+----+
MovieID|Cluster_Num|Movie_ID| Movie_Title| Movie_Genre|rank|
+-------+-----------+--------+--------------------+--------------------+----+
 3819| 0| 3819| Tampopo (1986)| Comedy| 1|
 3814| 0| 3814|Love and Death (1...| Comedy| 2|
 3812| 0| 3812|Everything You Al...| Comedy| 3|
 3811| 0| 3811|Breaker Morant (1...| Drama|War| 4|
 3801| 0| 3801|Anatomy of a Murd...| Drama|Mystery| 5|
 2300| 1| 2300|Producers, The (1...| Comedy|Musical| 1|
 1235| 1| 1235|Harold and Maude ...| Comedy| 2|
 1234| 1| 1234| Sting, The (1973)| Comedy|Crime| 3|
 1233| 1| 1233|Boat, The (Das Bo...| Action|Drama|War| 4|
 1230| 1| 1230| Annie Hall (1977)| Comedy|Romance| 5|
 3836| 2| 3836|Kelly's Heroes (1...| Action|Comedy|War| 1|
 3827| 2| 3827|Space Cowboys (2000)| Action|Sci-Fi| 2|
 3826| 2| 3826| Hollow Man (2000)|Horror|Sci-Fi|Thr...| 3|
 3809| 2| 3809|What About Bob? (...| Comedy| 4|
 3802| 2| 3802| Freejack (1992)| Action|Sci-Fi| 5|
 3755| 3| 3755|Perfect Storm, Th...|Action|Adventure|...| 1|
 3753| 3| 3753| Patriot, The (2000)| Action|Drama|War| 2|
 3535| 3| 3535|American Psycho (...|Comedy|Horror|Thr...| 3|
 3510| 3| 3510| Frequency (2000)| Drama|Thriller| 4|
 253| 3| 253|Interview with th...| Drama|Horror| 5|
+-------+-----------+--------+--------------------+--------------------+----+
only showing top 20 rows

In [0]:
display(Top5_Results_DF)

MovieID,Cluster_Num,Movie_ID,Movie_Title,Movie_Genre,rank
3819,0,3819,Tampopo (1986),Comedy,1
3814,0,3814,Love and Death (1975),Comedy,2
3812,0,3812,Everything You Always Wanted to Know About Sex (1972),Comedy,3
3811,0,3811,Breaker Morant (1980),Drama|War,4
3801,0,3801,Anatomy of a Murder (1959),Drama|Mystery,5
2300,1,2300,"Producers, The (1968)",Comedy|Musical,1
1235,1,1235,Harold and Maude (1971),Comedy,2
1234,1,1234,"Sting, The (1973)",Comedy|Crime,3
1233,1,1233,"Boat, The (Das Boot) (1981)",Action|Drama|War,4
1230,1,1230,Annie Hall (1977),Comedy|Romance,5


In [0]:
display(Results_DF)

MovieID,Cluster_Num,Movie_ID,Movie_Title,Movie_Genre
1718,9,1718,Stranger in the House (1997),Thriller
2031,9,2031,"$1,000,000 Duck (1971)",Children's|Comedy
1869,9,1869,Black Dog (1998),Action|Thriller
1868,9,1868,"Truce, The (1996)",Drama|War
854,9,854,"Ballad of Narayama, The (Narayama Bushiko) (1958)",Drama
344,2,344,Ace Ventura: Pet Detective (1994),Comedy
345,0,345,"Adventures of Priscilla, Queen of the Desert, The (1994)",Comedy|Drama
346,9,346,Backbeat (1993),Drama|Musical
347,9,347,Bitter Moon (1992),Drama
340,9,340,"War, The (1994)",Adventure|Drama


In [0]:
# output_FilePath = '/FileStore/tables/Output_Ques2_KMeans'
# Results_DF.repartition(1).write.csv(output_FilePath)